In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("Please switch to a GPU-enabled environment.")


GPU is available!


In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# # Load data
# df1 = pd.read_csv("https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/processed/reviews_cleaned_en.csv")
# df2 = pd.read_csv("https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/processed/reviews_cleaned_id.csv")

# # Combine the 'content' columns
# combined_content = pd.concat([df1['content'], df2['content']], ignore_index=True)

# # Make it a DataFrame
# df = pd.DataFrame({'content': combined_content})

# # Train-test split
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# print(train_df.shape, test_df.shape)


In [ ]:
# # --- Save to CSV ---
# train_df.to_csv("train_df.csv", index=False)
# test_df.to_csv("test_df.csv", index=False)

# # --- Load again ---

In [ ]:
# import pandas as pd
# train_df = pd.read_csv("train_df.csv")
# test_df = pd.read_csv("test_df.csv")

# print(train_df.shape, test_df.shape)

In [ ]:
# train_df, test_df = train_df[:1000], test_df[:1000]

In [ ]:
# from transformers import pipeline

# # --- Load pipelines with GPU ---
# aspect_pipeline = pipeline(
#     "token-classification",
#     model="Sengil/bert-based-aspect-extraction",
#     aggregation_strategy="max",
#     device=0  # GPU
# )

# sentiment_pipeline = pipeline(
#     "text-classification",
#     model="yangheng/deberta-v3-large-absa-v1.1",
#     device=0  # GPU
# )

# # --- Batch extraction function ---
# def extract_aspects_batch(texts, batch_size=32):
#     all_annotations = []
#     for i in range(0, len(texts), batch_size):
#         if (i % 20) == 0:
#             print(f'batch ke {i/batch_size}')
#         batch_texts = texts[i:i+batch_size]
#         # Extract aspects for the batch
#         # print(f'batch_texts : {batch_texts}')
#         batch_aspects = aspect_pipeline(batch_texts)
#         batch_annotations = []

#         for j, aspects in enumerate(batch_aspects):
#             text = batch_texts[j]
#             annotations = []
#             for r in aspects:
#                 term = r["word"]
#                 start = r["start"]
#                 end = r["end"]

#                 # Sentiment for each aspect in batch
#                 sentiment = sentiment_pipeline(term, text_pair=text)[0]
#                 annotations.append({
#                     "term": term,
#                     "from": start,
#                     "to": end,
#                     "sentiment": sentiment["label"],
#                     "score": sentiment["score"]
#                 })
#             text = text.split()
#             batch_annotations.append({"token": text, "aspects": annotations})
#         all_annotations.extend(batch_annotations)
#     return all_annotations

# # --- Train annotations ---
# train_df["annotations"] = extract_aspects_batch(train_df['content'].tolist())
# train_df["annotations"].to_json("absa_train.json", orient="records", indent=2, force_ascii=False)

# # --- Test annotations ---
# test_df["annotations"] = extract_aspects_batch(test_df['content'].tolist())
# test_df["annotations"].to_json("absa_test.json", orient="records", indent=2, force_ascii=False)

# print("✅ Done! Train and test annotations saved as JSON separately.")


# # --- Convert to dataframe ---
# # records = []
# # for ann in annotations:
# #     text = ann["text"]
# #     for asp in ann["aspects"]:
# #         records.append({
# #             "text": text,
# #             "aspect": asp["term"],
# #             "from": asp["from"],
# #             "to": asp["to"],
# #             "sentiment": asp["sentiment"],
# #             "score": asp["score"]
# #         })

# # absa_df = pd.DataFrame(records)
# # absa_df.to_csv("absa_model_output.csv", index=False)
# print("✅ Done! Batch ABSA extraction + sentiment prediction completed.")


In [ ]:
# from transformers import pipeline
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import math
# import json

# # --- Load pipelines with GPU ---
# aspect_pipeline = pipeline(
#     "token-classification",
#     model="Sengil/bert-based-aspect-extraction",
#     aggregation_strategy="max",
#     device=0  # GPU
# )

# device = "cuda" if torch.cuda.is_available() else "cpu"
# train_df = train_df.dropna()
# test_df = test_df.dropna()
# # --- Sentiment model ---
# sent_model_name = "yangheng/deberta-v3-base-absa-v1.1"  # base version to fit GPU
# sent_tokenizer = AutoTokenizer.from_pretrained(sent_model_name)
# sent_model = AutoModelForSequenceClassification.from_pretrained(sent_model_name).to(device)
# sent_model.eval()

# # --- Example batch ---
# batch_texts = [
#     "The food was great but the service was slow.",
#     "The battery life of this phone is terrible."
# ]

# # --- Batch extraction function ---
# def extract_aspects_batch(texts, batch_size=16):
#     all_annotations = []
#     for i in range(0, len(texts), batch_size):
#         if (i % 20) == 0:
#             print(f'batch ke {i/batch_size}')
#         batch_texts = texts[i:i+batch_size]
#         # Extract aspects for the batch
#         # print(f'batch_texts : {batch_texts}')
#         try:
#             batch_aspects = aspect_pipeline(batch_texts)
        

#             # --- Prepare batch for sentiment ---
#             sent_texts = []
#             sent_aspects = []
#             aspect_map = []  # to map back to original text/aspect
            
#             for i, (text, aspects) in enumerate(zip(batch_texts, batch_aspects)):
#                 for aspect in aspects:
#                     term = aspect["word"]
#                     sent_texts.append(text)
#                     sent_aspects.append(term)
#                     aspect_map.append(i)  # keep track of which text this aspect belongs to
            
#             # --- Tokenize for sentiment in batch ---
#             sent_inputs = sent_tokenizer(sent_texts, text_pair=sent_aspects, padding=True, truncation=True, return_tensors="pt").to(device)
            
#             # --- Forward pass sentiment model ---
#             with torch.no_grad():
#                 outputs = sent_model(**sent_inputs)
#                 probs = torch.softmax(outputs.logits, dim=-1)
#                 preds = torch.argmax(probs, dim=-1)
            
#            # --- Initialize batch_annotations for this batch ---
#             # batch_annotations = []
            
#             batch_annotations = [[] for _ in batch_texts]
    
#             for idx_in_flattened, text_idx in enumerate(aspect_map):
#                 batch_annotations[text_idx].append({
#                     "term": sent_aspects[idx_in_flattened],
#                     "sentiment": sent_model.config.id2label[preds[idx_in_flattened].item()],
#                     "score": probs[idx_in_flattened][preds[idx_in_flattened]].item()
#                 })
            
#             # Then add tokenized text
#             for i, ann in enumerate(batch_annotations):
#                 batch_annotations[i] = {"token": batch_texts[i].split(), "aspects": ann}
#             all_annotations.extend(batch_annotations)
#             # batch_annotations.append({
#             #     "token": text,
#             #     "aspects": ann
#             # })
#             torch.cuda.empty_cache()
#             del sent_inputs, outputs, probs, preds
#         except:
#             continue
        
#     return all_annotations

# # --- Helper to process and save in chunks ---
# def process_and_save(df, col_name, output_json, chunk_size=16000):
#     all_annotations = []
#     total = len(df)
#     num_chunks = math.ceil(total / chunk_size)

#     for i in range(6, num_chunks):
#         start = i * chunk_size
#         end = min(start + chunk_size, total)
#         print(f"Processing chunk {i+1}/{num_chunks} ({start}-{end})")

#         token_val = [str(t) for t in df[col_name].iloc[start:end].values]
#         chunk_annotations = extract_aspects_batch(token_val)
#         all_annotations.extend(chunk_annotations)

#         # Save after each chunk to avoid losing progress
#         with open(output_json, "w", encoding="utf-8") as f:
#             json.dump(all_annotations, f, indent=2, ensure_ascii=False)

#     return all_annotations

# # --- Train annotations ---
# # train_annotations = process_and_save(train_df, "content", "absa_train.json", chunk_size=16000)
# # train_df["annotations"] = train_annotations

# # # --- Test annotations ---
# # test_annotations = process_and_save(test_df, "content", "absa_test.json", chunk_size=16000)
# # test_df["annotations"] = test_annotations
# # ions ---

# # --- Train annotations ---
# train_texts = [str(i) for i in train_df["content"].values]
# train_annotations = extract_aspects_batch(train_texts)
# with open("dataset/absa_train.json", "w", encoding="utf-8") as f:
#     json.dump(train_annotations, f, indent=2, ensure_ascii=False)

# # --- Test annotations ---
# test_texts = [str(i) for i in test_df["content"].values]
# test_annotations = extract_aspects_batch(test_texts)
# with open("dataset/absa_test.json", "w", encoding="utf-8") as f:
#     json.dump(test_annotations, f, indent=2, ensure_ascii=False)

# print("✅ Done! Train and test ABSA annotations saved to 'dataset/' folder.")



# --- Convert to dataframe ---
# records = []
# for ann in annotations:
#     text = ann["text"]
#     for asp in ann["aspects"]:
#         records.append({
#             "text": text,
#             "aspect": asp["term"],
#             "from": asp["from"],
#             "to": asp["to"],
#             "sentiment": asp["sentiment"],
#             "score": asp["score"]
#         })

# absa_df = pd.DataFrame(records)
# absa_df.to_csv("absa_model_output.csv", index=False)
print("✅ Done! Batch ABSA extraction + sentiment prediction completed.")


In [ ]:
# import os, json, math

# def process_and_save(df, col_name, output_json, chunk_size=16000, resume_from=6):
#     all_annotations = []
#     total = len(df)
#     num_chunks = math.ceil(total / chunk_size)

#     # --- Resume from file if it exists ---
#     if os.path.exists(output_json):
#         with open(output_json, "r", encoding="utf-8") as f:
#             all_annotations = json.load(f)
#         if len(all_annotations) // chunk_size > resume_from:
#             resume_from = len(all_annotations) // chunk_size
#         print(f"🔄 Resuming from chunk {resume_from + 1}/{num_chunks}")
#     else:
#         print(f"▶️ Starting from chunk {resume_from + 1}/{num_chunks}")

#     # --- Main loop ---
#     for i in range(resume_from, num_chunks):
#         start = i * chunk_size
#         end = min(start + chunk_size, total)
#         print(f"Processing chunk {i+1}/{num_chunks} ({start}-{end})")

#         # Get text batch
#         token_val = [str(t) for t in df[col_name].iloc[start:end].values]

#         # Extract aspects
#         chunk_annotations = extract_aspects_batch(token_val)

#         # Ensure alignment
#         if len(chunk_annotations) != len(token_val):
#             print(f"⚠️ Warning: mismatch in chunk {i+1}, padding missing rows")
#             while len(chunk_annotations) < len(token_val):
#                 chunk_annotations.append([])

#         # Extend and save
#         all_annotations.extend(chunk_annotations)
#         with open(output_json, "w", encoding="utf-8") as f:
#             json.dump(all_annotations, f, indent=2, ensure_ascii=False)

#     print("✅ Processing complete.")
#     return all_annotations


# # --- Train annotations ---
# train_annotations = process_and_save(train_df, "content", "absa_train.json", chunk_size=16000)
# train_df["annotations"] = train_annotations

# # --- Test annotations ---
# test_annotations = process_and_save(test_df, "content", "absa_test.json", chunk_size=16000)
# test_df["annotations"] = test_a


In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# device = "cuda" if torch.cuda.is_available() else "cpu"

# # --- Sentiment model ---
# sent_model_name = "yangheng/deberta-v3-base-absa-v1.1"  # base version to fit GPU
# sent_tokenizer = AutoTokenizer.from_pretrained(sent_model_name)
# sent_model = AutoModelForSequenceClassification.from_pretrained(sent_model_name).to(device)
# sent_model.eval()

# # --- Example batch ---
# batch_texts = [
#     "The food was great but the service was slow.",
#     "The battery life of this phone is terrible."
# ]

# # --- Use your existing pipeline for aspect extraction ---
# batch_aspects = aspect_pipeline(batch_texts)  # list of list of dicts

# # --- Prepare batch for sentiment ---
# sent_texts = []
# sent_aspects = []
# aspect_map = []  # to map back to original text/aspect

# for i, (text, aspects) in enumerate(zip(batch_texts, batch_aspects)):
#     for aspect in aspects:
#         term = aspect["word"]
#         sent_texts.append(text)
#         sent_aspects.append(term)
#         aspect_map.append(i)  # keep track of which text this aspect belongs to

# # --- Tokenize for sentiment in batch ---
# sent_inputs = sent_tokenizer(sent_texts, text_pair=sent_aspects, padding=True, truncation=True, return_tensors="pt").to(device)

# # --- Forward pass sentiment model ---
# with torch.no_grad():
#     outputs = sent_model(**sent_inputs)
#     probs = torch.softmax(outputs.logits, dim=-1)
#     preds = torch.argmax(probs, dim=-1)

# # --- Collect results back into batch_texts structure ---
# batch_annotations = []
        
# for i, aspects in enumerate(batch_aspects):  # aspect_pipeline output
#     text = batch_texts[i].split()  # tokenized text
#     ann = []
#     for j, aspect in enumerate(aspects):
#         # Sentiment info comes from your batched model
#         ann.append({
#             "term": sent_aspects[i],  # from your batched sentiment list
#             "sentiment": sent_model.config.id2label[preds[i].item()],
#             "score": probs[i][preds[i]].item()
#         })
#     batch_annotations.append({
#         "token": text,
#         "aspects": ann
#     })

# print(batch_annotations)


In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch

# # Load tokenizer and model
# model_name = "yangheng/deberta-v3-large-absa-v1.1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model.eval()          # set model to evaluation mode (disables dropout, etc.)
# model.to("cuda")      # move model to GPU

# # Example batch
# texts = [
#     "The food was great but the service was slow.",
#     "The battery life of this phone is terrible."
# ]
# aspects = ["food", "battery"]

# # Tokenize the batch (text + text_pair)
# inputs = tokenizer(texts, text_pair=aspects, padding=True, truncation=True, return_tensors="pt").to("cuda")

# # Forward pass
# with torch.no_grad():
#     outputs = model(**inputs)
#     logits = outputs.logits

# # Convert logits to probabilities / predicted labels
# probs = torch.softmax(logits, dim=-1)
# preds = torch.argmax(probs, dim=-1)

# print(preds, probs)


In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# from datasets import load_dataset
# from transformers import AutoTokenizer

# # Load dataset
# dataset = load_dataset("lhoestq/conll2003")

# def tokenize_and_align_labels(batch, tokenizer):
#     print("\n=== NEW BATCH ===")
#     print("Original tokens example:")
#     print(batch["tokens"][0])  # show first example
#     print("Original NER tags example:")
#     print(batch["ner_tags"][0])

#     # Tokenize the batch
#     tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True)

#     print("\nTokenized input keys:", list(tokenized_inputs.keys()))
#     print("Example input_ids (first 20):", tokenized_inputs["input_ids"][0][:20])
#     print("Example tokens:", tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0][:20]))

#     labels = []
#     for i, label in enumerate(batch["ner_tags"]):
#         print(f'label : {label}')
#         word_ids = tokenized_inputs.word_ids(batch_index=i)
#         print(f"\nExample {i} word_ids:", word_ids[:20])

#         label_ids = []
#         previous_word_id = None
#         for word_id in word_ids:
#             if word_id is None:
#                 # label_ids.append(-100)
#                 label_ids.append(0)
#             elif word_id != previous_word_id:
#                 label_ids.append(label[word_id])
#             else:
#                 # same word split into multiple tokens, keep same label or -100
#                 label_ids.append(label[word_id])
#             previous_word_id = word_id

#         labels.append(label_ids)
#         print(f"Example {i} label_ids:", label_ids[:20])

#     tokenized_inputs["labels"] = labels
#     return tokenized_inputs

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# print()
# # Map the function to a small subset for clarity
# tokenized_dataset = dataset.map(
#     lambda x: tokenize_and_align_labels(x, tokenizer),
#     batched=True
# )



In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

# dataset = load_dataset(
#     "json",
#     data_files={
#         "train": r"D:\Code\Capstone\absa_model_output.json",
#     }
# )
dataset = load_dataset(
    "json",
    data_files={
        "train": r"D:\Code\Capstone\dataset\absa_train.json",
        "test":  r"D:\Code\Capstone\dataset\absa_test.json"
    }
)

def find_sequence(lst, seq):
    for i in range(len(lst) - len(seq) + 1):
        match = True
        for j in range(len(seq)):
            if seq[j] not in lst[i + j]:
                match = False
                break
        if match:
            return i, i + len(seq)
    return -1
    
def tokenize_and_align_labels(batch, tokenizer):
    print("\n" + "="*80)
    print("🆕 NEW BATCH")
    print("="*80)

    # Show first example info
    print(f"👉 Example tokens: {batch['token'][0]}")
    print(f"👉 Example aspects: {batch['aspects'][0]}")

    # Tokenize the batch
    tokenized_inputs = tokenizer(
        batch["token"],
        truncation=True,
        is_split_into_words=True
    )

    print("\n--- Tokenizer Output ---")
    print("Keys:", list(tokenized_inputs.keys()))
    print("Example input_ids (first 20):", tokenized_inputs["input_ids"][0])
    print("Example tokens (first 20):", tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0]))

    labels = []
    for i, tokens in enumerate(batch['token']):
        print("\n" + "-"*60)
        print(f"🔹 Sample {i}")
        print("-"*60)

        # Create default label 0 (O-tag)
        label = [0] * len(tokens)

        # Mark aspect terms with proper label IDs
        aspects = batch["aspects"][i]
        for aspect in aspects:
            # start_id, end_id = aspect['from'], aspect['to']
            term = aspect["term"].split()
            print(f'term : {term}')
            # print(f'find_sequence : {find_sequence(tokens, term)}')
            ind_term = find_sequence(tokens, term)
            print(f'ind term : {ind_term}')
            if ind_term == -1:
                continue
            start_id, end_id = ind_term
            polarity = aspect['sentiment']

            if polarity == 'Positive':
                label[start_id] = 1  # B-POS
                label[start_id + 1:end_id] = [2] * (end_id - start_id - 1)  # I-POS
            elif polarity == 'Negative':
                label[start_id] = 3  # B-NEG
                label[start_id + 1:end_id] = [4] * (end_id - start_id - 1)  # I-NEG
            elif polarity == 'Neutral':
                label[start_id] = 5  # B-NEU
                label[start_id + 1:end_id] = [6] * (end_id - start_id - 1)  # I-NEU

        print(f"🧩 Tokens: {tokens}")
        print(f"🏷️  Word-level labels: {label}")

        # Map labels to tokenized sequence
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        print(f"🔢 Word IDs (first 20): {word_ids[:20]}")

        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
                # Versu crf
                # label_ids.append(0)
            elif word_id != previous_word_id:
                label_ids.append(label[word_id])
            else:
                label_ids.append(label[word_id])
            previous_word_id = word_id

        labels.append(label_ids)

        # Print aligned results side by side for clarity
        token_strings = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i])
        pairs = list(zip(token_strings, label_ids))
        print("\n🧾 Token-label alignment:")
        for tok, lab in pairs:
            print(f"{tok:15} -> {lab}")
        
        print(f"\n✅ Final label_ids (first 20): {label_ids[:20]}")

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


# === Initialize tokenizer ===
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# === Map with debug printing ===
tokenized_dataset = dataset.map(
    lambda x: tokenize_and_align_labels(x, tokenizer),
    batched=True,
    # num_proc=2  # <- force single-process to see prints
)


D:\anaconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\capstone\Lib\site-packages\transformers\utils\hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# Total sample size
total_sample =5000

# Decide split ratio (e.g., 80% train, 20% test)
train_ratio = 0.8
train_sample_size = int(total_sample * train_ratio)  # 8000
test_sample_size  = total_sample - train_sample_size  # 2000

# Shuffle and select
sampled_train = tokenized_dataset["train"].shuffle(seed=42).select(range(train_sample_size))
sampled_test  = tokenized_dataset["test"].shuffle(seed=42).select(range(test_sample_size))

print(f"Train samples: {len(sampled_train)}, Test samples: {len(sampled_test)}")


Train samples: 4000, Test samples: 1000


In [4]:
# from datasets import load_dataset
# dataset = load_dataset(
#     "json",
#     data_files={
#         "train": r"D:\Code\Capstone\absa_model_output.json",
#     }
# )

In [5]:
# from datasets import load_dataset
# from transformers import AutoTokenizer

# # === Load dataset ===
# # dataset = load_dataset(
# #     "json",
# #     data_files={
# #         "train": r"D:\Code\Capstone\dataset\Stanford\Stanford\Laptops\train.json",
# #         "test": r"D:\Code\Capstone\dataset\Stanford\Stanford\Laptops\test.json",
# #     }
# # )

# def tokenize_and_align_labels(batch, tokenizer):
#     print("\n" + "="*80)
#     print("🆕 NEW BATCH")
#     print("="*80)

#     # Show first example info
#     print(f"👉 Example tokens: {batch['token'][0]}")
#     print(f"👉 Example aspects: {batch['aspects'][0]}")

#     # Tokenize the batch
#     tokenized_inputs = tokenizer(
#         batch["token"],
#         truncation=True,
#         is_split_into_words=True
#     )

#     print("\n--- Tokenizer Output ---")
#     print("Keys:", list(tokenized_inputs.keys()))
#     print("Example input_ids (first 20):", tokenized_inputs["input_ids"][0][:20])
#     print("Example tokens (first 20):", tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][0][:20]))

#     labels = []
#     for i, tokens in enumerate(batch['token']):
#         print("\n" + "-"*60)
#         print(f"🔹 Sample {i}")
#         print("-"*60)

#         # Create default label 0 (O-tag)
#         label = [0] * len(tokens)

#         # Mark aspect terms with proper label IDs
#         aspects = batch["aspects"][i]
#         for aspect in aspects:
#             start_id, end_id = aspect['from'], aspect['to']
#             polarity = aspect['polarity']

#             if polarity == 'positive':
#                 label[start_id] = 1  # B-POS
#                 label[start_id + 1:end_id] = [2] * (end_id - start_id - 1)  # I-POS
#             elif polarity == 'negative':
#                 label[start_id] = 3  # B-NEG
#                 label[start_id + 1:end_id] = [4] * (end_id - start_id - 1)  # I-NEG
#             elif polarity == 'neutral':
#                 label[start_id] = 5  # B-NEU
#                 label[start_id + 1:end_id] = [6] * (end_id - start_id - 1)  # I-NEU

#         print(f"🧩 Tokens: {tokens}")
#         print(f"🏷️  Word-level labels: {label}")

#         # Map labels to tokenized sequence
#         word_ids = tokenized_inputs.word_ids(batch_index=i)
#         print(f"🔢 Word IDs (first 20): {word_ids[:20]}")

#         label_ids = []
#         previous_word_id = None
#         for word_id in word_ids:
#             if word_id is None:
#                 label_ids.append(-100)
#                 # Versu crf
#                 # label_ids.append(0)
#             elif word_id != previous_word_id:
#                 label_ids.append(label[word_id])
#             else:
#                 label_ids.append(label[word_id])
#             previous_word_id = word_id

#         labels.append(label_ids)

#         # Print aligned results side by side for clarity
#         token_strings = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"][i])
#         pairs = list(zip(token_strings, label_ids))
#         print("\n🧾 Token-label alignment:")
#         for tok, lab in pairs:
#             print(f"{tok:15} -> {lab}")
        
#         print(f"\n✅ Final label_ids (first 20): {label_ids[:20]}")

#     tokenized_inputs["labels"] = labels
#     return tokenized_inputs


# # === Initialize tokenizer ===
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# # === Map with debug printing ===
# tokenized_dataset = dataset['train'].map(
#     lambda x: tokenize_and_align_labels(x, tokenizer),
#     batched=True
#     # num_proc=2  # <- force single-process to see prints
# )


In [6]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [7]:
import evaluate

seqeval = evaluate.load("seqeval")

In [8]:
import numpy as np

# labels = [label_list[i] for i in example[f"ner_tags"]]
    
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # true_predictions = [
    #     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    #     for prediction, label in zip(predictions, labels)
    # ]
    # true_labels = [
    #     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    #     for prediction, label in zip(predictions, labels)
    # ]
    true_labels = [
    [label_list[min(l, len(label_list)-1)] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
    ]

    true_predictions = [
    [label_list[min(p, len(label_list)-1)] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
    ]


    results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [9]:
# import numpy as np

# def compute_metrics(p):
#     predictions, labels = p
#     predictions = np.argmax(predictions, axis=2)

#     # Align predictions with labels (ignore -100)
#     true_predictions = [
#         [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     true_labels = [
#         [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
    
#     # Compute seqeval metrics
#     results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=0)

#     # Add per-class accuracy
#     per_class_acc = {}
#     for label in label_list:
#         # Flatten predictions and labels
#         flat_preds = [p for seq in true_predictions for p in seq]
#         flat_labels = [l for seq in true_labels for l in seq]
#         # Only consider current label
#         indices = [i for i, l in enumerate(flat_labels) if l == label]
#         if indices:
#             correct = sum(flat_preds[i] == flat_labels[i] for i in indices)
#             per_class_acc[label] = correct / len(indices)
#         else:
#             per_class_acc[label] = None  # No samples for this label

#     return {
#         "precision": results["overall_precision"],
#         "recall": results["overall_recall"],
#         "f1": results["overall_f1"],
#         "accuracy": results["overall_accuracy"],
#         "per_class_accuracy": per_class_acc
#     }


In [10]:
id2label = {
    0: "O",
    1: "B-positive",
    2: "I-positive",
    3: "B-negative",
    4: "I-negative",
    5: "B-neutral",
    6: "I-neutral",
}

label2id = {
    "O": 0,
    "B-positive": 1,
    "I-positive": 2,
    "B-negative": 3,
    "I-negative": 4,
    "B-neutral": 5,
    "I-neutral": 6,
}

label_list = [
    "O",
    "B-positive",
    "I-positive",
    "B-negative",
    "I-negative",
    "B-neutral",
    "I-neutral",
]


In [11]:
# # Untuk dataset conll2003
# id2label = {
#     0: "O",
#     1: "B-LOC",
#     2: "I-LOC",
#     3: "B-MISC",
#     4: "I-MISC",
#     5: "B-ORG",
#     6: "I-ORG",
#     7: "B-PER",
#     8: "I-PER"
# }

# label2id = {v: k for k, v in id2label.items()}

In [12]:
from collections import Counter

def get_label_ratio(dataset, id2label):
    # Flatten all label lists (each sample has a list of token labels)
    all_labels = [label for example in dataset["labels"] for label in example if label != -100]
    counts = Counter(all_labels)
    total = sum(counts.values())

    print("Label distribution:")
    for label_id, count in counts.items():
        label_name = id2label.get(label_id, str(label_id))
        print(f"{label_name:<15} {count:>5} ({count / total * 100:.2f}%)")

# Example usage:
get_label_ratio(tokenized_dataset['train'], id2label)

Label distribution:
O               3102703 (83.88%)
B-positive      256527 (6.94%)
B-neutral       73499 (1.99%)
I-neutral       29956 (0.81%)
I-positive      132288 (3.58%)
B-negative      72508 (1.96%)
I-negative      31328 (0.85%)


In [13]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch
# model = AutoModelForTokenClassification.from_pretrained(
#     "distilbert/distilbert-base-uncased", num_labels=7, id2label=id2label, label2id=label2id
# )
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased", num_labels=7, id2label=id2label, label2id=label2id
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# import numpy as np
# import warnings
# import logging
# print('hae')
# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=RuntimeWarning)
# warnings.filterwarnings("ignore", category=UserWarning, module="seqeval")
# warnings.filterwarnings('always')
# # Setup logger
# logging.basicConfig(
#     format='%(message)s',  # simple output (no timestamp)
#     level=logging.INFO
# )
# logger = logging.getLogger(__name__)

# # Define ranges
# lr_range = (1e-5, 5e-5)   # learning rate range
# wd_range = (0, 0.3)       # weight decay range
# num_runs = 100             # number of random combinations

# # Pre-generate random LRs and WDs
# log_lr_min, log_lr_max = np.log10(lr_range[0]), np.log10(lr_range[1])

# lr_list = []
# wd_list = []

# for _ in range(num_runs):
#     # Random LR in log scale
#     log_lr = np.random.uniform(log_lr_min, log_lr_max)
#     lr = 10 ** log_lr
#     lr_list.append(lr)

#     # Random weight decay in linear scale
#     wd = np.random.uniform(wd_range[0], wd_range[1])
#     wd_list.append(wd)
    
# for i in range(num_runs):
#     print('hae')
#     # Generate random values in log scale for lr (common practice)
#     lr = lr_list[i]
#     wd = wd_list[i]
#     # log_lr = np.random.uniform(-6, 1)
#     # log_reg = np.random.uniform(-5, 5)
    
#     # Convert to normal scale
#     # lr = 10 ** log_lr
#     # reg = 10 ** log_reg

#     # ✅ log info in one line like your print version
#     # logger.info(f"Run {i+1} || lr: {lr:.3e} || reg: {wd:.3e}")
#     print(f"Run {i+1}||", end='')
#     print(f"lr: {lr:.3e}||", end='')
#     print(f"reg: {wd:.3e}", end='')

#     training_args = TrainingArguments(
#         output_dir="my_awesome_wnut_model",
#         learning_rate=lr,
#         per_device_train_batch_size=16,
#         per_device_eval_batch_size=16,
#         num_train_epochs=2,
#         weight_decay=wd,
#         eval_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=False,
#         push_to_hub=False,
#     )
    
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_dataset["train"],
#         eval_dataset=tokenized_dataset["test"],
#         processing_class=tokenizer,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,
#     )
    
#     trainer.train()
#     print('hae')

In [15]:
# import numpy as np
# import warnings
# import logging
# print('hae')
# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=RuntimeWarning)
# warnings.filterwarnings("ignore", category=UserWarning, module="seqeval")
# warnings.filterwarnings('always')
# # Setup logger
# logging.basicConfig(
#     format='%(message)s',  # simple output (no timestamp)
#     level=logging.INFO
# )
# logger = logging.getLogger(__name__)

# # Define ranges
# lr_range = (1e-5, 4e-5)   # learning rate range
# wd_range = (0.006, 0.1)       # weight decay range
# num_runs = 100             # number of random combinations

# # Pre-generate random LRs and WDs
# log_lr_min, log_lr_max = np.log10(lr_range[0]), np.log10(lr_range[1])

# lr_list = []
# wd_list = []

# for _ in range(num_runs):
#     # Random LR in log scale
#     log_lr = np.random.uniform(log_lr_min, log_lr_max)
#     lr = 10 ** log_lr
#     lr_list.append(lr)

#     # Random weight decay in linear scale
#     wd = np.random.uniform(wd_range[0], wd_range[1])
#     wd_list.append(wd)
    
# for i in range(num_runs):
#     print('hae')
#     # Generate random values in log scale for lr (common practice)
#     lr = lr_list[i]
#     wd = wd_list[i]
#     # log_lr = np.random.uniform(-6, 1)
#     # log_reg = np.random.uniform(-5, 5)
    
#     # Convert to normal scale
#     # lr = 10 ** log_lr
#     # reg = 10 ** log_reg

#     # ✅ log info in one line like your print version
#     # logger.info(f"Run {i+1} || lr: {lr:.3e} || reg: {wd:.3e}")
#     print(f"Run {i+1}||", end='')
#     print(f"lr: {lr:.3e}||", end='')
#     print(f"reg: {wd:.3e}", end='')

#     training_args = TrainingArguments(
#         output_dir="my_awesome_wnut_model",
#         learning_rate=lr,
#         per_device_train_batch_size=16,
#         per_device_eval_batch_size=16,
#         num_train_epochs=10,
#         weight_decay=wd,
#         eval_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=False,
#         push_to_hub=False,
#     )
    
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_dataset["train"],
#         eval_dataset=tokenized_dataset["test"],
#         processing_class=tokenizer,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,
#     )
    
#     trainer.train()
#     print('hae')

In [16]:
# import torch
# from collections import Counter

# # your label distribution (ignore -100)
# counts = {
#     "O": 3102703,
#     "B-positive": 256527,
#     "B-neutral": 73499,
#     "I-neutral": 29956,
#     "I-positive": 132288,
#     "B-negative": 72508,
#     "I-negative": 31328,
# }

# # inverse frequency weights
# total = sum(counts.values())
# weights = {label: total / count for label, count in counts.items()}

# # normalize so weights roughly sum to num_labels (optional)
# norm_factor = sum(weights.values()) / len(weights)
# weights = {label: w / norm_factor for label, w in weights.items()}

# # convert to tensor (make sure order matches label2id)
# weights_tensor = torch.tensor([weights[label] for label in label2id.keys()], dtype=torch.float)
# print(weights_tensor)


In [17]:
import torch
from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Example weights: class 0 is 1, others 1–6 are weighted more
        class_weights = torch.tensor([0.0216, 0.2612, 0.5065, 0.9240, 2.1386, 0.9116, 2.2366]).to(logits.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
# 2.273e-05,
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=32,
#     num_train_epochs=1000,
#     weight_decay=1.037e-02,

In [18]:
# import numpy as np
# import warnings
# import logging
# from transformers import EarlyStoppingCallback

# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=RuntimeWarning)
# warnings.filterwarnings("ignore", category=UserWarning, module="seqeval")
# warnings.filterwarnings('always')

# def training_weighted(model, num_runs, epoch):
#     print('Start')
#     # Setup logger
#     logging.basicConfig(
#         format='%(message)s',  # simple output (no timestamp)
#         level=logging.INFO
#     )
#     logger = logging.getLogger(__name__)
    
#     # Define ranges
#     lr_range = (1e-5, 3e-5)     # learning rate
#     wd_range = (0.0, 0.05)      # weight decay
#     num_runs = 100             # number of random combinations
    
#     # Pre-generate random LRs and WDs
#     log_lr_min, log_lr_max = np.log10(lr_range[0]), np.log10(lr_range[1])
    
#     lr_list = []
#     wd_list = []
#     for _ in range(num_runs):
#            # Random LR in log scale
#         log_lr = np.random.uniform(log_lr_min, log_lr_max)
#         lr = 10 ** log_lr
#         lr_list.append(lr)
    
#         # Random weight decay in linear scale
#         wd = np.random.uniform(wd_range[0], wd_range[1])
#         wd_list.append(wd)

#     for i in range(num_runs):
#         lr = lr_list[i]
#         wd = wd_list[i]
#         lr = 1.509e-05
#         wd = 4.754e-02
#         # ✅ log info in one line like your print version
#         # logger.info(f"Run {i+1} || lr: {lr:.3e} || reg: {wd:.3e}")
#         print(f"Run {i+1}||", end='')
#         print(f"lr: {lr:.3e}||", end='')
#         print(f"reg: {wd:.3e}", end='')
    
        
#         training_args = TrainingArguments(
#             output_dir="my_awesome_wnut_model",
#             learning_rate=lr,
#             per_device_train_batch_size=16,
#             per_device_eval_batch_size=16,
#             num_train_epochs=epoch,
#             weight_decay=wd,
#             eval_strategy="epoch",
#             save_strategy="epoch",
#             metric_for_best_model="f1",
#             load_best_model_at_end=True,
#             push_to_hub=True,
#         )

#         early_stopping_callback = EarlyStoppingCallback(
#             early_stopping_patience=3,
#             early_stopping_threshold=0.01
#         )
    
#         trainer = WeightedTrainer(
#             model=model,
#             args=training_args,
#             train_dataset=tokenized_dataset['train'],
#             eval_dataset=tokenized_dataset['test'],
#             processing_class=tokenizer,
#             data_collator=data_collator,
#             compute_metrics=compute_metrics,
#             callbacks=[early_stopping_callback]
#         )
        
#         trainer.train()

# training_weighted(model=model, num_runs=1, epoch=10)

In [19]:
model_name = "bert-base-uncased"
def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, num_labels=7)

In [20]:
import optuna
from optuna.storages import RDBStorage

# Define persistent storage
storage = RDBStorage("sqlite:///optuna_trials.db")

study = optuna.create_study(
    study_name="transformers_optuna_studys1",
    direction="maximize",
    storage=storage,
    load_if_exists=True
)

[I 2025-10-22 12:46:27,885] Using an existing study with name 'transformers_optuna_studys1' instead of creating a new one.


In [21]:
import wandb
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import gc

# metric = evaluate.load("seqeval")

# def compute_metrics(eval_pred):
#     predictions = eval_pred.predictions.argmax(axis=-1)
#     labels = eval_pred.label_ids
#     return metric.compute(predictions=predictions, references=labels)

def compute_objective(metrics):
    gc.collect()
    torch.cuda.empty_cache()
    return metrics.get("f1", 0.0)  # fallback to 0 if 'f1' is missing

# def objective(trial):
#     # free memory at the start of each trial
#     gc.collect()
#     torch.cuda.empty_cache()

#     # define hyperparameters, create trainer, etc.
#     trainer = Trainer(
#         model_init=model_init,
#         args=training_args,
#         train_dataset=sampled_train,
#         eval_dataset=sampled_test,
#         tokenizer=tokenizer,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,
#     )

#     trainer.train()
#     metrics = trainer.evaluate()
#     return metrics["f1"]

wandb.init(project="hf-optuna", name="transformers_optuna_study")

# training_args = TrainingArguments(
#     output_dir="./results",
#         eval_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         logging_strategy="epoch",
#         num_train_epochs=3,
#         report_to="wandb",  # Logs to W&B
#         logging_dir="./logs",
#         run_name="transformers_optuna_study",
# )

training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",  # Logs to W&B
    logging_dir="./logs",
    run_name="transformers_optuna_study",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    push_to_hub=True,
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=sampled_train,
#     eval_dataset=sampled_test,
#     processing_class=tokenizer,
#     compute_metrics=compute_metrics,
# )
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=sampled_train,
    eval_dataset=sampled_test,
    data_collator=data_collator, 
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

D:\anaconda3\envs\capstone\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
D:\anaconda3\envs\capstone\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using t

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        # "per_device_train_batch_size": trial.suggest_categorical(
        #     "per_device_train_batch_size", [8, 16]
        # ),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
    }


best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=5,
    compute_objective=compute_objective,
    study_name="transformers_optuna_studys1",
    storage="sqlite:///optuna_trials.db",
    load_if_exists=True
)

print(best_run)

[I 2025-10-22 12:46:48,069] Using an existing study with name 'transformers_optuna_studys1' instead of creating a new one.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.734900,0.619805,0.379032,0.052765,0.092634,0.842621
2,0.629900,0.606348,0.365815,0.064272,0.109334,0.843657


[I 2025-10-22 13:06:03,102] Trial 6 finished with value: 0.0 and parameters: {'learning_rate': 1.7722789866867935e-06, 'weight_decay': 0.15735281405550236}. Best is trial 2 with value: 0.0.
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DB995E20F0>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 04f3d691-eacc-486a-92f5-7557ca41524f)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DCE43CEB70>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddri

eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,█▁
eval/recall,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁███
train/global_step,▁▁███
+3,...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.585100,0.505770,0.471585,0.242212,0.320045,0.860253
2,0.482100,0.486397,0.456592,0.278978,0.346341,0.863268


...

[I 2025-10-22 16:37:29,211] Trial 9 finished with value: 0.0 and parameters: {'learning_rate': 3.1066785998883832e-06, 'weight_decay': 0.1755409568247051}. Best is trial 2 with value: 0.0.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,█▁
eval/recall,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁███
train/global_step,▁▁███
+3,...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.666800,0.590182,0.356671,0.075779,0.125000,0.844692
2,0.591900,0.569631,0.374449,0.095425,0.152091,0.845849


[I 2025-10-22 16:56:52,586] Trial 10 finished with value: 0.0 and parameters: {'learning_rate': 4.602135761658058e-06, 'weight_decay': 0.1298642873123391}. Best is trial 2 with value: 0.0.


BestRun(run_id='2', objective=0.0, hyperparameters={'learning_rate': 1.9283721133035465e-06, 'weight_decay': 0.20220428470198348}, run_summary=None)


In [23]:
import subprocess

# Stop previous dashboard if running
# On Windows, just restart the kernel if needed

# Start dashboard in background
subprocess.Popen([
    "python", "-m", "optuna.dashboard",
    "sqlite:///optuna_trials.db",
    "--port", "8080"
])


<Popen: returncode: None args: ['python', '-m', 'optuna.dashboard', 'sqlite:...>

In [25]:
import optuna
from optuna.visualization.matplotlib import (
    plot_optimization_history,
    plot_intermediate_values,
    plot_param_importances
)
import matplotlib.pyplot as plt

# Load the study from RDB storage
storage = optuna.storages.RDBStorage("sqlite:///optuna_trials.db")

study = optuna.load_study(
    study_name="transformers_optuna_study",
    storage=storage
)

# Plot optimization history
ax1 = plot_optimization_history(study)
plt.show()
ax1.figure.savefig("optimization_history.png")

# Plot intermediate values (if using pruning and intermediate reports)
ax2 = plot_intermediate_values(study)
plt.show()
ax2.figure.savefig("intermediate_values.png")

# Plot parameter importances
ax3 = plot_param_importances(study)
plt.show()
ax3.figure.savefig("param_importances.png")

ModuleNotFoundError: No module named 'matplotlib'

In [81]:
import torch
import torch.nn as nn
from torchcrf import CRF
from transformers import DistilBertModel

class DistilBERT_CRF(nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained(pretrained_model_name)
        # self.hidden2tag = nn.Linear(self.bert.config.hidden_size, num_labels)
        # self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask=None, labels=None):
        print("🚀 Forward started")
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # sequence_output = outputs.last_hidden_state  # [batch, seq_len, hidden_size]
        # emissions = self.hidden2tag(sequence_output) # [batch, seq_len, num_labels]
        # print("✅ BERT + Linear done")

        # if labels is not None:
        #     # 🧠 Debug section
        #     print("⚙️  DEBUG:")
        #     print(f"emissions shape: {emissions.shape}")
        #     print(f"labels shape: {labels.shape}")
        #     print(f"mask shape: {attention_mask.shape}")
        #     print(f"Min label: {labels.min().item()}, Max label: {labels.max().item()}")
        #     print(f"Num labels in CRF: {self.crf.num_tags}")
        #     assert labels.shape == attention_mask.shape, "❌ Label and mask must have same shape"
        #     assert emissions.shape[:2] == labels.shape, "❌ Emission batch/seq must match labels"

        #     # Compute CRF loss
        #     loss = -self.crf(emissions, labels, mask=attention_mask.bool(), reduction='mean')
        #     return loss
        # else:
        #     # Decode tags
        #     prediction = self.crf.decode(emissions, mask=attention_mask.bool())
            # return prediction
        return outputs


In [82]:
model = DistilBERT_CRF("distilbert/distilbert-base-uncased", 7)

In [87]:
tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'labels']
)
print(type(labels), type(input_ids), type(attention_mask))


<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [90]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [94]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader
import torch

# === Load model & tokenizer ===
# model_name = "bert-base-uncased"  # change to your model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=7)

# === Example dataset ===
texts = ["I love PyTorch", "Transformers are powerful"]
labels = [[1, 2, 0, 0], [1, 0, 0]]  # dummy labels

# === Tokenize ===
tokenized_dataset = tokenizer(texts, truncation=True, padding=False, return_tensors=None)

# === Make PyTorch dataset ===
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

dataset = MyDataset(tokenized_dataset, labels)

# === DataLoader with automatic padding ===
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
loader = DataLoader(dataset, batch_size=2, collate_fn=data_collator)

# === Move model to device ===
device = 'cpu'
model.to(device)
model.eval()

# === Run single batch forward ===
for batch in loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    print("\n🔹 Batch shapes:", {k: v.shape for k, v in batch.items()})
    print("Device:", next(model.parameters()).device)

    with torch.no_grad():
        try:
            output = model(**batch)
            print("✅ Forward successful")
            if hasattr(output, "loss"):
                print(f"Loss: {output.loss.item()}")
        except Exception as e:
            print("❌ Forward failed:", e)
    break


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [78]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [41]:
import numpy as np
import warnings
import logging
print('hae')
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="seqeval")
warnings.filterwarnings('always')
# Setup logger
logging.basicConfig(
    format='%(message)s',  # simple output (no timestamp)
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Define ranges
# lr_range = (1e-5, 4e-5)   # learning rate range
# wd_range = (0.006, 0.1)       # weight decay range
# num_runs = 100             # number of random combinations

# # Pre-generate random LRs and WDs
# log_lr_min, log_lr_max = np.log10(lr_range[0]), np.log10(lr_range[1])

# lr_list = []
# wd_list = []
device = torch.device("cpu")
model.to(device)


# # Generate random values in log scale for lr (common practice)
# lr = lr_list[i]
# wd = wd_list[i]
# log_lr = np.random.uniform(-6, 1)
# log_reg = np.random.uniform(-5, 5)

# Convert to normal scale
# lr = 10 ** log_lr
# reg = 10 ** log_reg

# ✅ log info in one line like your print version
# logger.info(f"Run {i+1} || lr: {lr:.3e} || reg: {wd:.3e}")
# print(f"Run {i+1}||", end='')
# print(f"lr: {lr:.3e}||", end='')
# print(f"reg: {wd:.3e}", end='')

training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    no_cuda=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
print('hae')

hae


Epoch,Training Loss,Validation Loss


IndexError: list index out of range

In [60]:
tokenized_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'labels']
)

In [56]:
from datasets import Dataset
from transformers import DataCollatorForTokenClassification

# ensure your tokenized dataset outputs torch tensors
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# data collator for padding
data_collator = DataCollatorForTokenClassification(tokenizer)


In [57]:
print(f"Model device: {next(model.parameters()).device}")
batch = next(iter(tokenized_dataset['train']))
print({k: v.device for k, v in batch.items() if hasattr(v, 'device')})


Model device: cpu
{'input_ids': device(type='cpu'), 'attention_mask': device(type='cpu'), 'labels': device(type='cpu')}


In [25]:
log_lr = np.random.uniform(-10, 5)
log_reg = np.random.uniform(-10, 5)
lr = 10 ** log_lr
reg = 10 ** log_reg

# ✅ log info in one line like your print version
logger.info(f"Run {1} || lr: {lr:.3e} || reg: {reg:.3e}")
# print(f"Run {i+1}||", end='')
print(f"lr: {lr:.3e}||", end='')
print(f"reg: {reg:.3e}", end='')

Run 1 || lr: 5.014e-02 || reg: 7.788e-08


lr: 5.014e-02||reg: 7.788e-08

In [36]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.372000,0.363314,0.531576,0.409221,0.462442,0.881093
2,0.333900,0.344586,0.539962,0.450871,0.491411,0.885402


TrainOutput(global_step=13632, training_loss=0.3793022414328347, metrics={'train_runtime': 8083.401, 'train_samples_per_second': 26.983, 'train_steps_per_second': 1.686, 'total_flos': 1.2816425860625376e+16, 'train_loss': 0.3793022414328347, 'epoch': 2.0})

In [11]:
tokenized_dataset

Dataset({
    features: ['token', 'aspects', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [ ]:
trainer.push_to_hub()